In [1]:
import edward as ed
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
%matplotlib inline

Image Model:

$$I(x,y) =  \sum_{i=0}^{N_{basis}-1} \beta_i \psi(x,y)$$
where
$$
\psi_i(x,y) = \exp\left\{\ \left[\begin{array}{c}x - C_x^i\\
                                     y - C_y^i\end{array}\right]^T \Sigma^{-1}\left[\begin{array}{c}x - C_x^i\\
                                     y - C_y^i\end{array}\right]\right\}
$$

$$
\Sigma = \left[\begin{array}{cc}\sigma_1^2   & \sigma_{12} \\
                                 \sigma_{12} & \sigma_2^2 \end{array}\right]
$$
Visibility Model:

$$ V(u,v) = \sum_{i=1}^{N_{B}} \alpha_i \phi(u,v) $$

$$\phi(u,v) = \exp\left\{\frac{av^2+bu^2-cuv}{4ab-c^2} + j(fv+eu)\right\}$$
$$\alpha_i = \frac{\beta_i}{\sqrt{4ab-c^2}}$$
$$ a = -\frac{1}{2}\frac{\sigma_2^2}{\sigma_1^2\sigma_2^2 - \sigma_{12}^2} ; b = -\frac{1}{2}\frac{\sigma_1^2}{\sigma_1^2\sigma_2^2 - \sigma_{12}^2}; c = \frac{\sigma_{12}}{\sigma_1^2\sigma_2^2 - \sigma_{12}^2} $$
$$e = C_x^i; f = C_y^i$$

```
FourierTransform[exp( (a*(x-e)^2+b*(y-f)^2+c*(x-e)*(y-f))),{x,y},{u,v}]
```

In [2]:
def test():
    sess = tf.test.TestCase()
    with sess.test_session():
        x = tf.square([2, 3])
        sess.assertAllEqual(x.eval(), [4, 9])
        return x.eval()
test()

array([4, 9], dtype=int32)

In [25]:
def tf_phi(U,a,b,c,e,f):
    u = U[:,0]
    v = U[:,1]
    
    A = tf.exp(tf.divide(a*v**2 + b*u**2 - c*u*v, 4.*tf.multiply(a,b)-tf.multiply(c,c)))
    sinPart = tf.sin(f*v+e*u)
    cosPart = tf.cos(f*v+e*u)
    return A*cosPart, A*sinPart

In [26]:
def visibility_model(U,alpha,a,b,c,e,f, factor = None, IMAGE_SIZE_X = 128, IMAGE_SIZE_Y = 128):
    
    realPart, imagPart = tf_phi(U,a,b,c,e,f)
    
    return tf.reduce_sum(tf.multiply(alpha,realPart), axis=1), tf.reduce_sum(tf.multiply(alpha,imagPart), axis=1)

In [30]:
u = np.pi*np.ones([5,1])
v = 0.*np.ones([5,1])
U = np.stack([u,v],axis=1)

a = np.array([1,1,1]).astype('float64')
b = np.array([1,1,1]).astype('float64')
c = np.array([1,1,1]).astype('float64')
e = np.array([1,1,1]).astype('float64')
f = np.array([1,1,1]).astype('float64')
alpha = np.array([1,1,1], dtype=np.float64)

a_tf = tf.constant(a)
b_tf = tf.constant(b)
c_tf = tf.constant(c)
e_tf = tf.constant(e)
f_tf = tf.constant(f)
alpha_tf = tf.constant(alpha)

with tf.Session() as sess:
    realPart, imagPart = visibility_model(U,alpha,a,b,c,e,f)
    print(realPart.eval() + 1j*imagPart.eval())

[-80.51797266 +9.86060775e-15j -80.51797266 +9.86060775e-15j
 -80.51797266 +9.86060775e-15j -80.51797266 +9.86060775e-15j
 -80.51797266 +9.86060775e-15j]


In [31]:
def phi(u,v,a,b,c,e,f):
    return np.exp((a*v**2 + b*u**2 - c*u*v)/(4.*a*b-c**2) + 1j*(f*v+e*u))

In [32]:
np.sum(alpha*phi(u,v,a,b,c,e,f), axis = 1)

array([-80.51797266 +9.86060775e-15j, -80.51797266 +9.86060775e-15j,
       -80.51797266 +9.86060775e-15j, -80.51797266 +9.86060775e-15j,
       -80.51797266 +9.86060775e-15j])